Queremos disponer de un direccionario de tipos concretos en los que las claves van a ser números enteros y los valores van a ser instancias de la clase Cliente.

In [81]:
class Cliente:
    def __init__(self, nombre: str, apellidos: str, dni: str):
        self.nombre = nombre
        self.apellidos = apellidos
        self.dni = dni

    def __repr__(self):
        return f"Cliente(Nombre: {self.nombre}, Apellidos: {self.apellidos}, DNI: {self.dni})"
    
class DictTipado(dict):
    key_type = str
    value_type = str

    @classmethod
    def validate_items(self, items: dict):
        return all(self.validate_item(k,v) for k, v in items.items())
    
    @classmethod
    def validate_item(self, k, v):
        if not isinstance(k, self.key_type) :
            raise TypeError('Clave incorrecta')
        
        if not isinstance(v, self.value_type):
            raise TypeError('Tipo incorrecto')


    def __init__(self, initial: dict = {}):
        self.validate_items(initial)
        super().__init__(initial)

    def __setitem__(self, key, value):
        self.validate_item(key, value)
        return super().__setitem__(key, value)

class ListaClientes(DictTipado):
    key_type = int
    value_type = Cliente



In [82]:
lc = ListaClientes({1: Cliente('Martín', 'Molina', '1234568A')})
lc[2] = Cliente('Juan', 'Magán', '1234568B')

In [83]:
ListaClientes()

{}

In [84]:
ListaClientes({1: 1})

TypeError: Tipo incorrecto

In [51]:
ListaClientes({'a': Cliente('Martin', 'Molina', '12345678A')})

TypeError: Clave incorrecta

In [34]:
# Ejemplo de uso
cliente1 = Cliente("Juan", "Fernandez", "12345678A")
cliente2 = Cliente("María", "Pérez", "12345678B")

# Diccionario con claves enteras y valores de tipo Cliente
diccionario_clientes = {1: cliente1, 2: cliente2}

# Validación
resultado = ListaClientes.validate(diccionario_clientes)
print("¿Todos los clientes son válidos?", resultado)  # Salida: True

¿Todos los clientes son válidos? True


In [ ]:
lc = ListaClientes({1: Cliente('Martín', 'Molina', '1234568A')})
lc[2] = Cliente('Juan', 'Magán', '1234568B')

TypeError: keywords must be strings

In [89]:
def dict_tipado(key_type, value_type):
    """
    Decorador que crea un diccionario tipado con tipos específicos para claves y valores.
    
    Args:
        key_type: El tipo que deben tener las claves del diccionario
        value_type: El tipo que deben tener los valores del diccionario
    
    Returns:
        Una clase que hereda de dict con validación de tipos
    """
    def decorator(cls):
        # Creamos una nueva clase que hereda tanto de dict como de la clase decorada
        class DictWrapper(dict):
            def validate_item(self, key, value):
                if not isinstance(key, key_type):
                    raise TypeError(f'Clave incorrecta. Se esperaba {key_type.__name__} pero se obtuvo {type(key).__name__}')
                
                if not isinstance(value, value_type):
                    raise TypeError(f'Valor incorrecto. Se esperaba {value_type.__name__} pero se obtuvo {type(value).__name__}')
                return True
            
            def validate_items(self, items):
                return all(self.validate_item(k, v) for k, v in items.items())
            
            def __init__(self, initial=None):
                if initial is None:
                    initial = {}
                self.validate_items(initial)
                dict.__init__(self, initial)
                
                # Inicializar atributos de la clase decorada si los tiene
                if hasattr(cls, '__init__'):
                    # Llamamos al __init__ de la clase decorada sin pasar argumentos
                    # que ya hemos procesado para el diccionario
                    cls_instance = cls.__new__(cls)
                    try:
                        cls.__init__(cls_instance)
                    except:
                        pass
            
            def __setitem__(self, key, value):
                self.validate_item(key, value)
                return dict.__setitem__(self, key, value)
            
            @classmethod
            def validate(cls, dictionary):
                try:
                    for key, value in dictionary.items():
                        if not isinstance(key, key_type) or not isinstance(value, value_type):
                            return False
                    return True
                except:
                    return False
        
        # Copiamos los métodos y atributos de la clase original
        for attr_name, attr_value in cls.__dict__.items():
            if not attr_name.startswith('__'):
                setattr(DictWrapper, attr_name, attr_value)
        
        # Establecemos los tipos en la clase resultante
        DictWrapper.key_type = key_type
        DictWrapper.value_type = value_type
        
        # Actualizamos el nombre y documentación para que parezca la clase original
        DictWrapper.__name__ = cls.__name__
        DictWrapper.__qualname__ = cls.__qualname__
        DictWrapper.__doc__ = cls.__doc__
        
        return DictWrapper
    
    return decorator

In [91]:
@dict_tipado(int, Cliente)
class ListaClientesAnotacion(dict):
    """Diccionario que solo acepta claves enteras y valores de tipo Cliente"""
    pass

In [92]:
ldc = ListaClientesAnotacion({1: Cliente('Martín', 'Molina', '1234568A')})
ldc[2] = Cliente('Juan', 'Magán', '1234568B')
print(lc) 

{1: Cliente(Nombre: Martín, Apellidos: Molina, DNI: 1234568A), 2: Cliente(Nombre: Juan, Apellidos: Magán, DNI: 1234568B)}
